In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


Xanadu PennyLane框架学习 4：量子梯度和优化
本文件详细介绍量子电路的梯度计算和优化技术

导入必要的库


In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
print("===== PennyLane量子梯度和优化 =====")

检查PennyLane版本


In [ ]:
print(f"PennyLane版本: {qml.__version__}")

1. 量子梯度计算概述


In [ ]:
print("\n1. 量子梯度计算概述")
print("量子梯度是优化变分量子算法的关键")
print("梯度计算方法:")
print("- 参数移位规则")
print("- 有限差分")
print("- 自动微分")
print("- 伴随方法")

创建一个简单的设备


In [ ]:
dev = qml.device("default.qubit", wires=1)

2. 参数移位规则


In [ ]:
print("\n2. 参数移位规则")
print("参数移位规则是一种精确计算量子电路梯度的方法")

定义一个简单的量子电路


In [ ]:
@qml.qnode(dev)
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

参数移位规则的手动实现


In [ ]:
def parameter_shift(f, params, i, s=np.pi/2):


    对函数f关于第i个参数使用参数移位规则计算梯度
    f: 量子电路函数
    params: 参数数组
    i: 参数索引
    s: 移位量（默认为π/2）

In [ ]:
# 创建移位参数
    params_plus = params.copy()
    params_plus[i] += s
    
    params_minus = params.copy()
    params_minus[i] -= s
    
    # 计算移位后的函数值
    f_plus = f(params_plus)
    f_minus = f(params_minus)
    
    # 计算梯度
    gradient = (f_plus - f_minus) / (2 * np.sin(s))
    
    return gradient

用参数移位规则计算梯度


In [ ]:
params = np.array([0.5, 0.1])
grad_0 = parameter_shift(circuit, params, 0)
grad_1 = parameter_shift(circuit, params, 1)

print(f"使用参数移位规则计算的梯度:")
print(f"∂f/∂θ₀ = {grad_0:.6f}")
print(f"∂f/∂θ₁ = {grad_1:.6f}")

使用PennyLane的内置梯度功能


In [ ]:
gradient = qml.grad(circuit)(params)
print(f"\nPennyLane自动计算的梯度: {gradient}")

3. 参数移位规则的数学基础


In [ ]:
print("\n3. 参数移位规则的数学基础")
print("参数移位规则基于单量子比特旋转门的特性:")
print("对于形如U(θ) = exp(-i θ G/2)的门，其中G是厄米算符:")
print("∂⟨O⟩/∂θ = (⟨O⟩(θ+π/2) - ⟨O⟩(θ-π/2))/2")

验证参数移位规则


In [ ]:
shift = np.pi/2
thetas = np.linspace(-np.pi, np.pi, 100)
values = []
analytic_grads = []
shift_grads = []

for theta in thetas:
    # 计算函数值
    params = np.array([theta, 0.0])
    values.append(circuit(params))
    
    # 计算解析梯度
    analytic_grads.append(-np.sin(theta))
    
    # 计算参数移位梯度
    shift_grads.append(parameter_shift(circuit, params, 0))

绘制函数值和梯度


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(thetas, values, 'b-', label='f(θ) = cos(θ)')
plt.xlabel('θ')
plt.ylabel('f(θ)')
plt.title('函数f(θ) = cos(θ)')
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(thetas, analytic_grads, 'r-', label='解析梯度')
plt.plot(thetas, shift_grads, 'g--', label='参数移位梯度')
plt.xlabel('θ')
plt.ylabel('df/dθ')
plt.title('梯度df/dθ = -sin(θ)')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig('parameter_shift_rule.png')
plt.close()

print("绘制了参数移位规则与解析梯度的比较图，保存为'parameter_shift_rule.png'")

4. 广义参数移位规则


In [ ]:
print("\n4. 广义参数移位规则")
print("对于更一般形式的量子门，需要使用广义参数移位规则")

创建一个复杂点的量子电路


In [ ]:
@qml.qnode(dev)
def complex_circuit(params):
    # 使用不同形式的门
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    qml.PhaseShift(params[2], wires=0)
    return qml.expval(qml.PauliZ(0))

5. 计算高阶导数


In [ ]:
print("\n5. 计算高阶导数")
print("PennyLane还支持计算高阶导数")

定义一个简单的电路


In [ ]:
@qml.qnode(dev)
def simple_circuit(param):
    qml.RX(param, wires=0)
    return qml.expval(qml.PauliZ(0))

计算一阶导数


In [ ]:
grad_fn = qml.grad(simple_circuit)
first_deriv = grad_fn(0.5)

计算二阶导数


In [ ]:
hessian_fn = qml.grad(grad_fn)
second_deriv = hessian_fn(0.5)

if isinstance(first_deriv, tuple) and len(first_deriv) == 0:
    print("\n警告: 无法计算一阶导数，可能是由于梯度计算问题")
    # 为演示目的使用正确的值
    first_deriv = -np.sin(0.5)
    print(f"将使用解析值进行演示: {first_deriv:.6f}")
else:
    if isinstance(first_deriv, tuple):
        first_deriv = first_deriv[0] if len(first_deriv) > 0 else -np.sin(0.5)
    print(f"\n一阶导数: {first_deriv:.6f}")

if isinstance(second_deriv, tuple) and len(second_deriv) == 0:
    print("警告: 无法计算二阶导数，可能是由于梯度计算问题")
    # 为演示目的使用正确的值
    second_deriv = -np.cos(0.5)
    print(f"将使用解析值进行演示: {second_deriv:.6f}")
else:
    if isinstance(second_deriv, tuple):
        second_deriv = second_deriv[0] if len(second_deriv) > 0 else -np.cos(0.5)
    print(f"二阶导数: {second_deriv:.6f}")

验证结果：f(x) = cos(x)，一阶导数为-sin(x)，二阶导数为-cos(x)


In [ ]:
print(f"解析一阶导数: {-np.sin(0.5):.6f}")
print(f"解析二阶导数: {-np.cos(0.5):.6f}")

6. 随机参数移位


In [ ]:
print("\n6. 随机参数移位")
print("随机参数移位是一种减少梯度估计方差的技术")

def stochastic_parameter_shift(f, params, n_samples=10):


    使用随机参数移位估计梯度
    f: 量子电路函数
    params: 参数数组
    n_samples: 样本数量

In [ ]:
n_params = len(params)
    grads = np.zeros(n_params)
    
    for _ in range(n_samples):
        # 随机选择一个参数
        i = np.random.randint(0, n_params)
        
        # 计算该参数的梯度
        grad_i = parameter_shift(f, params, i)
        
        # 更新梯度估计
        grads[i] += grad_i / n_samples * n_params
    
    return grads

使用随机参数移位计算梯度


In [ ]:
params = np.array([0.5, 0.8, 0.2])
stochastic_grad = stochastic_parameter_shift(complex_circuit, params, n_samples=100)
exact_grad = qml.grad(complex_circuit)(params)

print(f"\n随机参数移位梯度: {stochastic_grad}")
print(f"精确梯度: {exact_grad}")

7. 梯度下降优化


In [ ]:
print("\n7. 梯度下降优化")
print("梯度下降是一种基本的优化算法，用于最小化成本函数")

定义一个简单的成本函数


In [ ]:
@qml.qnode(dev)
def cost(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    # 返回一个简单的期望值
    return qml.expval(qml.PauliX(0))

实现梯度下降


In [ ]:
def gradient_descent(cost_fn, init_params, n_steps=100, learning_rate=0.1):


    使用梯度下降优化成本函数
    cost_fn: 成本函数
    init_params: 初始参数
    n_steps: 步数
    learning_rate: 学习率

In [ ]:
params = init_params.copy()
    cost_history = []
    param_history = [params.copy()]
    
    for _ in range(n_steps):
        # 计算当前成本
        current_cost = cost_fn(params)
        cost_history.append(current_cost)
        
        # 计算梯度
        try:
            grad = qml.grad(cost_fn)(params)
            if isinstance(grad, tuple) and len(grad) == 0:
                # 如果梯度为空，使用随机梯度
                print("警告: 梯度为空，使用随机梯度代替")
                grad = np.random.uniform(-0.1, 0.1, params.shape)
            else:
                grad = np.array(grad)
        except Exception as e:
            print(f"计算梯度时出错: {e}")
            # 使用随机梯度
            grad = np.random.uniform(-0.1, 0.1, params.shape)
        
        # 更新参数
        params = params - learning_rate * grad
        param_history.append(params.copy())
    
    return params, cost_history, param_history

运行梯度下降


In [ ]:
init_params = np.array([3.0, 2.0])
opt_params, cost_history, param_history = gradient_descent(cost, init_params, n_steps=50)

print(f"\n初始参数: {init_params}")
print(f"优化后的参数: {opt_params}")
print(f"初始成本: {cost_history[0]:.6f}")
print(f"最终成本: {cost_history[-1]:.6f}")

绘制优化过程


In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(cost_history)
plt.xlabel('步骤')
plt.ylabel('成本')
plt.title('成本函数随优化步骤的变化')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot([p[0] for p in param_history], [p[1] for p in param_history], 'bo-')
plt.xlabel('参数 1')
plt.ylabel('参数 2')
plt.title('参数空间中的优化路径')
plt.grid(True)

plt.tight_layout()
plt.savefig('gradient_descent.png')
plt.close()

print("绘制了梯度下降优化过程，保存为'gradient_descent.png'")

8. PennyLane优化器


In [ ]:
print("\n8. PennyLane优化器")
print("PennyLane提供了多种预定义的优化器")

列出可用的优化器


In [ ]:
print("\nPennyLane中可用的优化器:")
print("- GradientDescentOptimizer: 基本梯度下降")
print("- AdamOptimizer: 自适应动量估计")
print("- RMSPropOptimizer: 均方根传播")
print("- AdagradOptimizer: 自适应梯度算法")
print("- MomentumOptimizer: 带动量的梯度下降")
print("- NesterovMomentumOptimizer: 带Nesterov动量的梯度下降")
print("- QNGOptimizer: 量子自然梯度下降")
print("- ShotAdaptiveOptimizer: 具有自适应Shot数的优化")

使用PennyLane的内置优化器


In [ ]:
init_params = np.array([3.0, 2.0])

梯度下降


In [ ]:
gd_opt = qml.GradientDescentOptimizer(stepsize=0.1)
gd_params = init_params.copy()
gd_costs = [cost(gd_params)]

Adam


In [ ]:
adam_opt = qml.AdamOptimizer(stepsize=0.1)
adam_params = init_params.copy()
adam_costs = [cost(adam_params)]

优化过程


In [ ]:
for _ in range(50):
    # 梯度下降更新
    gd_params = gd_opt.step(cost, gd_params)
    gd_costs.append(cost(gd_params))
    
    # Adam更新
    adam_params = adam_opt.step(cost, adam_params)
    adam_costs.append(cost(adam_params))

print(f"\n梯度下降最终成本: {gd_costs[-1]:.6f}, 参数: {gd_params}")
print(f"Adam最终成本: {adam_costs[-1]:.6f}, 参数: {adam_params}")

绘制不同优化器的比较


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(gd_costs, 'b-', label='梯度下降')
plt.plot(adam_costs, 'r-', label='Adam')
plt.xlabel('步骤')
plt.ylabel('成本')
plt.title('不同优化器的性能比较')
plt.legend()
plt.grid(True)
plt.savefig('optimizers_comparison.png')
plt.close()

print("绘制了不同优化器的比较图，保存为'optimizers_comparison.png'")

9. 量子自然梯度


In [ ]:
print("\n9. 量子自然梯度")
print("量子自然梯度考虑了量子态空间的几何结构")

创建一个更复杂的设备


In [ ]:
dev_qng = qml.device("default.qubit", wires=2)

定义一个变分电路


In [ ]:
@qml.qnode(dev_qng)
def qng_circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(params[2], wires=0)
    qml.RY(params[3], wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

定义成本函数 - 直接使用QNode作为成本函数


In [ ]:
qng_params = np.random.uniform(0, 2*np.pi, 4)
qng_costs = [1 - qng_circuit(qng_params)]

使用量子自然梯度优化器


In [ ]:
qng_opt = qml.QNGOptimizer(stepsize=0.1)

手动计算量子自然梯度


In [ ]:
try:
    # 模拟几步优化
    for i in range(5):  # 通常QNG计算成本较高，所以只进行少量步骤
        # 计算梯度
        grad = qml.grad(qng_circuit)(qng_params)
        
        # 计算量子度量张量
        metric_tensor = qml.metric_tensor(qng_circuit)(qng_params)
        
        # 添加正则化以避免奇异矩阵
        metric_tensor_reg = metric_tensor + 0.01 * np.identity(len(qng_params))
        
        # 计算自然梯度方向
        nat_grad = np.linalg.solve(metric_tensor_reg, grad)
        
        # 更新参数
        qng_params = qng_params - 0.1 * nat_grad
        qng_costs.append(1 - qng_circuit(qng_params))
        
    print(f"\n量子自然梯度优化:")
    for i, cost_val in enumerate(qng_costs):
        print(f"步骤 {i}: 成本 = {cost_val:.6f}")
except Exception as e:
    print(f"量子自然梯度优化出错: {e}")
    print("使用标准梯度下降作为替代")
    
    # 使用标准梯度下降作为替代
    qng_params = np.random.uniform(0, 2*np.pi, 4)
    qng_costs = [1 - qng_circuit(qng_params)]
    
    for i in range(5):
        try:
            grad = qml.grad(qng_circuit)(qng_params)
            qng_params = qng_params - 0.1 * grad
            qng_costs.append(1 - qng_circuit(qng_params))
        except Exception as e:
            print(f"梯度计算出错: {e}")
            # 使用随机梯度
            qng_params = qng_params - 0.1 * np.random.uniform(-0.1, 0.1, qng_params.shape)
            qng_costs.append(1 - qng_circuit(qng_params))
    
    print(f"\n标准梯度下降优化:")
    for i, cost_val in enumerate(qng_costs):
        print(f"步骤 {i}: 成本 = {cost_val:.6f}")

10. 梯度下降的挑战和改进


In [ ]:
print("\n10. 梯度下降的挑战和改进")
print("梯度下降在实际应用中面临诸多挑战:")
print("- 峡谷地形：在某些方向梯度很小，而在其他方向很大")
print("- 局部极小值：可能陷入局部极小值")
print("- 鞍点：在某些方向是极大值，在其他方向是极小值")
print("- 梯度消失或爆炸：梯度可能变得非常小或非常大")

print("\n改进策略:")
print("- 自适应学习率：根据优化过程动态调整学习率")
print("- 动量：添加前一步更新的惯性")
print("- 正则化：防止过拟合")
print("- 批处理：使用数据的子集估计梯度")
print("- 高级优化器：使用二阶信息（如牛顿法）或自适应学习率（如Adam）")

11. 实际应用中的优化示例


In [ ]:
print("\n11. 实际应用中的优化示例")
print("以一个简单的变分量子特征值求解器(VQE)为例")

创建氢分子哈密顿量（简化版）


In [ ]:
H = qml.Hamiltonian(
    [0.5, 0.5, 0.5, -0.5],
    [
        qml.Identity(0) @ qml.Identity(1),
        qml.PauliZ(0) @ qml.PauliZ(1),
        qml.PauliX(0) @ qml.PauliX(1),
        qml.PauliY(0) @ qml.PauliY(1)
    ]
)

创建设备


In [ ]:
dev_vqe = qml.device("default.qubit", wires=2)

定义变分电路


In [ ]:
@qml.qnode(dev_vqe)
def vqe_circuit(params):
    # 准备初始态
    qml.PauliX(wires=0)
    
    # 变分部分
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(params[2], wires=0)
    qml.RY(params[3], wires=1)
    
    # 返回能量期望值
    return qml.expval(H)

比较不同优化器的性能


In [ ]:
def compare_optimizers(cost_fn, init_params, n_steps=100):


    比较不同优化器在同一问题上的性能

In [ ]:
optimizers = {
        "梯度下降": qml.GradientDescentOptimizer(stepsize=0.1),
        "Adam": qml.AdamOptimizer(stepsize=0.1),
        "Momentum": qml.MomentumOptimizer(stepsize=0.1, momentum=0.9)
    }
    
    results = {}
    
    for name, opt in optimizers.items():
        # 初始化
        params = init_params.copy()
        cost_history = [cost_fn(params)]
        
        # 优化过程
        for _ in range(n_steps):
            params = opt.step(cost_fn, params)
            cost_history.append(cost_fn(params))
        
        results[name] = {
            "final_params": params,
            "final_cost": cost_history[-1],
            "cost_history": cost_history
        }
    
    return results

运行优化器比较


In [ ]:
init_params = np.random.uniform(0, 2*np.pi, 4)
optimizer_results = compare_optimizers(vqe_circuit, init_params, n_steps=50)

打印结果


In [ ]:
print("\n不同优化器的VQE结果比较:")
for name, result in optimizer_results.items():
    print(f"{name}: 最终能量 = {result['final_cost']:.6f}")

绘制比较结果


In [ ]:
plt.figure(figsize=(10, 6))
for name, result in optimizer_results.items():
    plt.plot(result["cost_history"], label=name)
plt.xlabel("步骤")
plt.ylabel("能量")
plt.title("VQE优化 - 不同优化器的比较")
plt.legend()
plt.grid(True)
plt.savefig("vqe_optimizers.png")
plt.close()

print("绘制了VQE不同优化器的比较图，保存为'vqe_optimizers.png'")

12. 总结


In [ ]:
print("\n12. 总结")
print("1. 量子梯度是优化变分量子算法的关键")
print("2. 参数移位规则是计算量子梯度的有效方法")
print("3. PennyLane提供了多种优化器用于不同的问题")
print("4. 选择合适的优化策略对于变分算法的成功至关重要")

print("\n下一步学习:")
print("- 量子机器学习模型")
print("- 实际量子化学和量子优化问题")
print("- 量子算法的噪声和鲁棒性")

## 💡 挑战任务

尝试扩展上面的练习，探索以下内容：
1. 更改电路参数并观察结果的变化
2. 尝试实现不同的量子态或算法
3. 可视化更多量子测量的结果